In [222]:
import random
from bs4 import BeautifulSoup
import re
import requests
import pandas as pd
import numpy as np
import math
import random
import unidecode
import datetime
from time import strptime

# Below are dicts, lists, and strings that will be used in later functions
header_name = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36'
acronym_to_city_dict = {'ATL': 'Atlanta',
                        'WSH': 'Washington',
                        'NYM': 'New York Mets',
                        'PHI': 'Philadelphia',
                        'MIA': 'Miami',
                        'STL': 'St. Louis',
                        'MIL': 'Milwaukee',
                        'CHC': 'Chicago Cubs',
                        'CIN': 'Cincinatti',
                        'PIT': 'Pittsburg',
                        'LAD': 'Los Angeles Dodgers',
                        'ARI': 'Arizona',
                        'SF': 'San Francisco',
                        'COL': 'Colorado',
                        'SD': 'San Diego',
                        'NYY': 'New York Yankees',
                        'TB': 'Tampa Bay',
                        'BOS': 'Boston',
                        'TOR': 'Toronto',
                        'BAL': 'Baltimore',
                        'MIN': 'Minnesota',
                        'CLE': 'Cleveland',
                        'CHW': 'Chicago White Sox',
                        'KC': 'Kansas City',
                        'DET': 'Detroit',
                        'HOU': 'Houston',
                        'OAK': 'Oakland',
                        'TEX': 'Texas',
                        'LAA': 'Los Angeles Angels',
                        'SEA': 'Seattle'}
#acronym_to_city_dict = {'LAA': 'Los Angeles Angels',
  #                      'Hou': 'Houston'}
nickname_to_city_dict = {'braves': 'Atlanta',
                        'nationals': 'Washington',
                        'mets': 'New York Mets',
                        'phillies': 'Philadelphia',
                        'marlins': 'Miami',
                        'cardinals': 'St. Louis',
                        'brewers': 'Milwaukee',
                        'cubs': 'Chicago Cubs',
                        'reds': 'Cincinatti',
                        'pirates': 'Pittsburg',
                        'dodgers': 'Los Angeles Dodgers',
                        'diamondbacks': 'Arizona',
                        'd-backs': 'Arizona',
                        'giants': 'San Francisco',
                        'rockies': 'Colorado',
                        'padres': 'San Diego',
                        'yankees': 'New York Yankees',
                        'rays': 'Tampa Bay',
                        'red sox': 'Boston',
                        'blue jays': 'Toronto',
                        'orioles': 'Baltimore',
                        'twins': 'Minnesota',
                        'indians': 'Cleveland',
                        'white sox': 'Chicago White Sox',
                        'royals': 'Kansas City',
                        'tigers': 'Detroit',
                        'astros': 'Houston',
                        'athletics': 'Oakland',
                        'rangers': 'Texas',
                        'angels': 'Los Angeles Angels',
                        'mariners': 'Seattle'}
league = {'Atlanta': 'NL',
          'Washington': 'NL',
          'New York Mets': 'NL',
          'Philadelphia': 'NL',
          'Miami': 'NL',
          'St. Louis': 'NL',
          'Milwaukee': 'NL',
          'Chicago Cubs': 'NL',
          'Cincinatti': 'NL',
          'Pittsburg': 'NL',
          'Los Angeles Dodgers': 'NL',
          'Arizona': 'NL',
          'San Francisco': 'NL',
          'Colorado': 'NL',
          'San Diego': 'NL',
          'New York Yankees': 'AL',
          'Tampa Bay': 'AL',
          'Boston': 'AL',
          'Toronto': 'AL',
          'Baltimore': 'AL',
          'Minnesota': 'AL',
          'Cleveland': 'AL',
          'Chicago White Sox': 'AL',
          'Kansas City': 'AL',
          'Detroit': 'AL',
          'Houston': 'AL',
          'Oakland': 'AL',
          'Texas': 'AL',
          'Los Angeles Angels': 'AL',
          'Seattle': 'AL'}
mlb_teams = ['braves', 'nationals', 'mets', 'phillies', 'marlins', 
         'cardinals', 'brewers', 'cubs', 'reds', 'pirates', 
         'dodgers', 'd-backs', 'giants', 'rockies', 'padres', 
         'yankees', 'rays', 'red sox', 'blue jays', 'orioles', 'twins',
         'indians', 'white sox', 'royals', 'tigers', 'astros', 'athletics',
         'rangers', 'angels', 'mariners']

In [231]:
class Batter:
    """Class that holds information for a Batter.
    
    Attributes:

    """
    def __init__(self, team, name, hand, singles_L, doubles_L, triples_L, home_runs_L, walks_L, so_L, singles_R, 
                 doubles_R, triples_R, home_runs_R, walks_R, so_R, sb_rate, sb_success):
        """Initializes values for this class
        
        Args:
            team: team name of batter
            name: batter name
            hand: Batting hand of hitter(L, R, or S)
            singles_L: fraction of plate appearances ending in a single vs. lefties
            doubles_L: fraction of plate appearances ending in a double vs. lefties
            triples_L: fraction of plate appearances ending in a triple vs. lefties
            home_runs_L: fraction of plate appearances ending in a home run vs. lefties
            walks_L: fraction of plate appearances ending in a walk vs. lefties
            so_L: fraction of plate appearances ending in a strikeout vs. lefties
            singles_RL: fraction of plate appearances ending in a single vs. righties
            doubles_R: fraction of plate appearances ending in a double vs. righties
            triples_R: fraction of plate appearances ending in a triple vs. righties
            home_runs_R: fraction of plate appearances ending in a home run vs. righties
            walks_R: fraction of plate appearances ending in a walk vs. righties
            so_R: fraction of plate appearances ending in a strikeout vs. righties
            sb_rate: approximate percentage of time player will steal base given opportunity
            sb_success: fraction of stolen base attempts ending in success
        """
        self.team = team
        self.name = name
        self.hand = hand
        self.singles_L = singles_L
        self.doubles_L = singles_L + doubles_L
        self.triples_L = singles_L + doubles_L + triples_L
        self.home_runs_L = singles_L + doubles_L + triples_L + home_runs_L
        self.walks_L = singles_L + doubles_L + triples_L + home_runs_L + walks_L
        self.so_L = singles_L + doubles_L + triples_L + home_runs_L + walks_L + so_L
        self.singles_R = singles_R
        self.doubles_R = singles_R + doubles_R
        self.triples_R = singles_R + doubles_R + triples_R
        self.home_runs_R = singles_R + doubles_R + triples_R + home_runs_R
        self.walks_R = singles_R + doubles_R + triples_R + home_runs_R + walks_R
        self.so_R = singles_R + doubles_R + triples_R + home_runs_R + walks_R + so_R
        self.sb_rate = sb_rate
        self.sb_success = sb_success
        
        #self.cum values represent cumulative (season) totals for a batter
        self.cum_singles_L = 0
        self.cum_doubles_L = 0
        self.cum_triples_L = 0
        self.cum_home_runs_L = 0
        self.cum_walks_L = 0
        self.cum_so_L = 0
        self.cum_pa_L = 0
        self.cum_singles_R = 0
        self.cum_doubles_R = 0
        self.cum_triples_R = 0
        self.cum_home_runs_R = 0
        self.cum_walks_R = 0
        self.cum_so_R = 0
        self.cum_pa_R = 0
        self.cum_runs = 0
        self.cum_rbi = 0
        self.cum_sb = 0
        self.cum_cs = 0
        self.cum_gdp = 0
     
    def hits(self):
        """cumulative hits of a batter
        
        Returns:
            int: number of cumulative hits
        """
        return(self.cum_singles_L+self.cum_doubles_L+self.cum_triples_L+self.cum_home_runs_L+
               self.cum_singles_R+self.cum_doubles_R+self.cum_triples_R+self.cum_home_runs_R)
    
    def avg(self):
        """Calculates avg of a player
        
        Returns:
            float: batting average in decimal form
        """
        if((self.cum_pa_L+self.cum_pa_R)-(self.cum_walks_L+self.cum_walks_R)>0):
            return(round(float((self.hits())/((self.cum_pa_L+self.cum_pa_R)-(self.cum_walks_L+self.cum_walks_R))), 3))
        else:
            return 0    
        
    def obp(self):
        """Calculates on base percentage of a player
        
        Returns:
            float: opb in decimal form
        """
        if((self.cum_pa_L+self.cum_pa_R)>0):
            return(round((self.hits()+(self.cum_walks_L+self.cum_walks_R))/(self.cum_pa_L+self.cum_pa_R), 3))
        else:
            return 0
        
    def slg(self):
        """Calculates slugging percentage of a player
        
        Returns:
            float: slg in decimal form
        """
        if((self.cum_pa_L+self.cum_pa_R)-(self.cum_walks_L+self.cum_walks_R)>0):
            return(round(((self.cum_singles_L+self.cum_singles_R)+2*(self.cum_doubles_L+self.cum_doubles_R)+
                           3*(self.cum_triples_L+self.cum_triples_R)+4*(self.cum_home_runs_L+self.cum_home_runs_R))/
                           ((self.cum_pa_L+self.cum_pa_R)-(self.cum_walks_L+self.cum_walks_R)), 3))
        else:
            return 0
        
    def ops(self):
        """Calculates on base plus slugging of a player
        
        Returns:
            float: ops in decimal form
            """
        return(self.slg() + self.obp())
    
    def strikeouts(self):
        """Number of strikeouts of a batter
        
        Returns:
            float: total strikeouts
        """
        return(self.cum_so_L+self.cum_so_R)
    
    def print_stats(self):
        """Prints some stats of a player
        """
        print("Avg: " + str(round(self.avg(), 3)))
        print("OBP: " + str(round(self.obp(), 3)))
        print("SLG: " + str(round(self.slg(), 3)))
        print("OPS: " + str(round(self.ops(), 3)))
        print("HR: " + str((self.cum_home_runs_L+self.cum_home_runs_R)))
              
    def hits_L(self):
        """cumulative hits of a batter against lefties
        
        Returns:
            int: number of hits of batter agianst lefties
        """
        return(self.cum_singles_L+self.cum_doubles_L+self.cum_triples_L+self.cum_home_runs_L)
    
    def avg_L(self):
        """Calculates avg of a player against lefties
        
        Returns:
            float: batting average in decimal form of batter against lefties
        """
        if((self.cum_pa_L-self.cum_walks_L)>0):
            return(round(float((self.hits_L())/(self.cum_pa_L-self.cum_walks_L)), 3))
        else:
            return 0    
              
    def obp_L(self):
        """Calculates on base percentage of a player against lefties
        
        Returns:
            float: opb in decimal form of batter against lefties
        """
        if(self.cum_pa_L>0):
            return(round((self.hits_L()+self.cum_walks_L)/self.cum_pa_L, 3))
        else:
            return 0
              
    def slg_L(self):
        """Calculates slugging percentage of a player against lefties
        
        Returns:
            float: slg in decimal form of batter against lefties
            """
        if((self.cum_pa_L-self.cum_walks_L)>0):
            return(round(((self.cum_singles_L+2*self.cum_doubles_L+
                           3*self.cum_triples_L+4*self.cum_home_runs_L))/
                           (self.cum_pa_L-self.cum_walks_L), 3))
        else:
            return 0
              
    def ops_L(self):
        """Calculates on base plus slugging of a player against lefties
        
        Returns:
            float: ops in decimal form of batter against lefties
        """
        return(self.slg_L() + self.obp_L())
              
    def hits_R(self):
        """Cumulative hits of a batter against righties
        
        Returns:
            int: number of hits of batter against righties
        """
        return(self.cum_singles_R+self.cum_doubles_R+self.cum_triples_R+self.cum_home_runs_R)
    
    def avg_R(self):
        """Calculates avg of a player against righties
        
        Returns:
            float: batting average in decimal form of batter against righties
        """
        if((self.cum_pa_R-self.cum_walks_R)>0):
            return(round(float((self.hits_R())/(self.cum_pa_R-self.cum_walks_R)), 3))
        else:
            return 0    
              
    def obp_R(self):
        """Calculates on base percentage of a player against righties
        
        Returns:
            float: opb in decimal form of batter against righties
        """
        if(self.cum_pa_R>0):
            return(round((self.hits_R()+self.cum_walks_R)/self.cum_pa_R, 3))
        else:
            return 0
              
    def slg_R(self):
        """Calculates slugging percentage of a player against righties
        
        Returns:
            float: slg in decimal form of batter against righties
        """
        if((self.cum_pa_R-self.cum_walks_R)>0):
            return(round(((self.cum_singles_R2*self.cum_doubles_R+
                           3*self.cum_triples_R+4*self.cum_home_runs_R))/
                           (self.cum_pa_R-self.cum_walks_R), 3))
        else:
            return 0
              
    def ops_R(self):
        """Calculates on base plus slugging of a player against righties
        
        Returns:
            float: ops in decimal form of batter against righties
        """
        
        return(self.slg_R() + self.obp_R())
              
class Pitcher(Batter):
    """Class that holds information for a pitcher. Many attributes are similar to those from the Batter class.
    
    Attributes:

    """
    def __init__(self, team, name, hand, singles_L, doubles_L, triples_L, home_runs_L, walks_L, so_L, singles_R,
                 doubles_R, triples_R, home_runs_R, walks_R, so_R, sb_rate, sb_success):
        """Initializes values for this class
        
        Args:
            team: team name of pitcher
            name: pitcher name
            hand: Pitching arm of pitcher(L, R)
            singles_L: fraction of plate appearances ending in a single vs. lefties
            doubles_L: fraction of plate appearances ending in a double vs. lefties
            triples_L: fraction of plate appearances ending in a triple vs. lefties
            home_runs_L: fraction of plate appearances ending in a home run vs. lefties
            walks_L: fraction of plate appearances ending in a walk vs. lefties
            singles_RL: fraction of plate appearances ending in a single vs. righties
            doubles_R: fraction of plate appearances ending in a double vs. righties
            triples_R: fraction of plate appearances ending in a triple vs. righties
            home_runs_R: fraction of plate appearances ending in a home run vs. righties
            walks_R: fraction of plate appearances ending in a walk vs. righties
            sb_rate: approximate percentage of time opposing player will steal base given opportunity (set to 0)
            sb_success: fraction of opponent stolen base attempts ending in success (set to 0)
        """
        
        Batter.__init__(self, team, name, hand, singles_L, doubles_L, triples_L, home_runs_L, walks_L, so_L,
                        singles_R, doubles_R, triples_R, home_runs_R, walks_R, so_R, sb_rate, sb_success)
        self.cum_outs_L = 0
        self.cum_outs_R = 0
        self.current_bf = 0
        self.current_outs = 0
        self.current_runs = 0
        self.finished_half = False
    
    def cum_outs(self, hand):
        """Cumulative outs of a pitcher against batters of certain handedness
        
        Args:
            hand: handedness of batter pitcher is facing
        Returns:
            float: value for cumulative outs
        """
        if(hand=='L'):
            return self.cum_outs_L
        elif(hand=='R'):
            return self.cum_outs_R
    
    def ip(self):
        """Calculates innings pitched of a pitcher
        
        Returns:
            float: Innings pitched in normal form(ie. 5.1 for 5 and a third innings pitched)
        """
        ip = (self.cum_outs_L+self.cum_outs_R+self.cum_cs)/3.0
        ip = math.modf(ip)
        ip = round((ip[0]/3.33333 + ip[1]), 2)
        return(ip)  
    
    def era(self):
        """Calculates ERA of a pitcher
        
        Returns:
            float: ERA in decimal form
        """
        if((self.cum_outs_L+self.cum_outs_R)==0):
            return 0
        else:
            return(round((27*self.cum_runs/(self.cum_outs_L+self.cum_outs_R+self.cum_cs)), 2))
    
    def whip(self):
        """Calculates WHIP of a pitcher
        
        Returns:
            float: WHIP in decimal form
        """
        if((self.cum_outs_L+self.cum_outs_R)==0):
            return 0
        else:
            return(round((((self.cum_walks_L+self.cum_walks_R)+self.hits())/
                          ((self.cum_outs_L+self.cum_outs_R+self.cum_cs)/3)), 2))
        
    

class Team:
    """Class that holds information for a team.
    
    Attributes:

    """
    def __init__(self, city, league, batter_df, pitcher_df, starters, relievers_df):
        """Initializes values for this class
        
        Args:
            city: city of team (ex. 'Atlanta')
            league: league of team ('AL' or 'NL')
            batter_df: dataframe of batters on a team and their 2019 stats
            pitcher_df: dataframe of pitchers on a team and their 2019 stats
            starters: dict with names of batters in projected starting lineup, rotation, and the closer
            relievers_df: dataframe with relievers of a team and their projected innings pitched
        """
        self.city = city
        self.league = league
        self.batter_df = batter_df
        self.pitcher_df = pitcher_df
        self.starters = starters
        self.relievers_df = relievers_df
        self.pitching_staff = pitcher_df
        self.closer = None
        self.current_pitcher_list = []
        self.available_pitcher_list = []
        self.current_pitcher = None
        self.batting_index = 0
        self.rotation_index = 0
        self.runners = [None, None, None, 0]    # first base-home plate. 0 for base empty, 1 for runner on base
        self.runs = 0
        self.wins = 0
        self.losses = 0
        self.cum_runs = 0
        self.cum_runs_allowed = 0
        self.pitchers = []
        self.batters = []
        self.game_no = 0

        avg_stats = self.batter_df.loc[batter_df['Name']=='Avg_Totals']
        avg_singles_L = avg_stats['1B_L'].values[0]/avg_stats['PA_L'].values[0]
        avg_doubles_L = avg_stats['2B_L'].values[0]/avg_stats['PA_L'].values[0]
        avg_triples_L = avg_stats['3B_L'].values[0]/avg_stats['PA_L'].values[0]
        avg_home_runs_L = avg_stats['HR_L'].values[0]/avg_stats['PA_L'].values[0]
        avg_walks_L = avg_stats['BB_L'].values[0]/avg_stats['PA_L'].values[0]
        avg_so_L = avg_stats['SO_L'].values[0]/avg_stats['PA_L'].values[0]
        avg_singles_R = avg_stats['1B_R'].values[0]/avg_stats['PA_R'].values[0]
        avg_doubles_R = avg_stats['2B_R'].values[0]/avg_stats['PA_R'].values[0]
        avg_triples_R = avg_stats['3B_R'].values[0]/avg_stats['PA_R'].values[0]
        avg_home_runs_R = avg_stats['HR_R'].values[0]/avg_stats['PA_R'].values[0]
        avg_walks_R = avg_stats['BB_R'].values[0]/avg_stats['PA_R'].values[0]
        avg_so_R = avg_stats['SO_R'].values[0]/avg_stats['PA_R'].values[0]
        batter_df = batter_df[batter_df['Name']!='Avg_Totals']
        
        for index, batter in batter_df.iterrows():   # adding values to each batter and instantiating Batter class
            singles_L = max(0, round(2*batter['1B_L']/batter['PA_L']-avg_singles_L, 3))
            doubles_L = max(0, round(2*batter['2B_L']/batter['PA_L']-avg_doubles_L, 3))
            triples_L = max(0, round(2*batter['3B_L']/batter['PA_L']-avg_triples_L, 3))
            home_runs_L = max(0, round(2*batter['HR_L']/batter['PA_L']-avg_home_runs_L, 3))
            walks_L = max(0, round(2*batter['BB_L']/batter['PA_L']-avg_walks_L, 3))
            so_L = max(0, round(2*batter['SO_L']/batter['PA_L']-avg_so_L, 3))
            singles_R = max(0, round(2*batter['1B_R']/batter['PA_R']-avg_singles_R, 3))
            doubles_R = max(0, round(2*batter['2B_R']/batter['PA_R']-avg_doubles_R, 3))
            triples_R = max(0, round(2*batter['3B_R']/batter['PA_R']-avg_triples_R, 3))
            home_runs_R = max(0, round(2*batter['HR_R']/batter['PA_R']-avg_home_runs_R, 3))
            walks_R = max(0, round(2*batter['BB_R']/batter['PA_R']-avg_walks_R, 3))
            so_R = max(0, round(2*batter['SO_R']/batter['PA_R']-avg_so_R, 3))

            sb_rate = round((batter['SB_L']+batter['SB_R']+batter['CS_L']+batter['CS_R'])/
                            (batter['1B_L']+batter['1B_R']+batter['BB_L']+batter['BB_R']), 3)
            if(batter['CS_L']+batter['CS_R'])==0:
                sb_success = 0
            else:
                sb_success = ((batter['SB_L']+batter['SB_R'])/
                             (batter['SB_L']+batter['SB_R']+batter['CS_L']+batter['CS_R']))

            self.batters.append(Batter(self.city, batter['Name'], batter['Bats'], singles_L, doubles_L, triples_L, 
                                       home_runs_L, walks_L, so_L, singles_R, doubles_R, triples_R, 
                                       home_runs_R, walks_R, so_R, sb_rate, sb_success))
            
        avg_stats = self.pitcher_df.loc[pitcher_df['Name']=='Avg_Totals']
        avg_singles_L = avg_stats['1B_L'].values[0]/avg_stats['TBF_L'].values[0]
        avg_doubles_L = avg_stats['2B_L'].values[0]/avg_stats['TBF_L'].values[0]
        avg_triples_L = avg_stats['3B_L'].values[0]/avg_stats['TBF_L'].values[0]
        avg_home_runs_L = avg_stats['HR_L'].values[0]/avg_stats['TBF_L'].values[0]
        avg_walks_L = avg_stats['BB_L'].values[0]/avg_stats['TBF_L'].values[0]
        avg_so_L = avg_stats['SO_L'].values[0]/avg_stats['TBF_L'].values[0]
        avg_singles_R = avg_stats['1B_R'].values[0]/avg_stats['TBF_R'].values[0]
        avg_doubles_R = avg_stats['2B_R'].values[0]/avg_stats['TBF_R'].values[0]
        avg_triples_R = avg_stats['3B_R'].values[0]/avg_stats['TBF_R'].values[0]
        avg_home_runs_R = avg_stats['HR_R'].values[0]/avg_stats['TBF_R'].values[0]
        avg_walks_R = avg_stats['BB_R'].values[0]/avg_stats['TBF_R'].values[0]
        avg_so_R = avg_stats['SO_R'].values[0]/avg_stats['TBF_R'].values[0]
        pitcher_df = pitcher_df[pitcher_df['Name']!='Avg_Totals']
        for index, pitcher in pitcher_df.iterrows(): # adding values to each pitcher and instantiating Pitcher class
            singles_L = max(0, round(2*pitcher['1B_L']/pitcher['TBF_L']-avg_singles_L, 3))
            doubles_L = max(0, round(2*pitcher['2B_L']/pitcher['TBF_L']-avg_doubles_L, 3))
            triples_L = max(0, round(2*pitcher['3B_L']/pitcher['TBF_L']-avg_triples_L, 3))
            home_runs_L = max(0, round(2*pitcher['HR_L']/pitcher['TBF_L']-avg_home_runs_L, 3))
            walks_L = max(0, round(2*pitcher['BB_L']/pitcher['TBF_L']-avg_walks_L, 3))
            so_L = max(0, round(2*pitcher['SO_L']/pitcher['TBF_L']-avg_so_L, 3))
            
            singles_R = max(0, round(2*pitcher['1B_R']/pitcher['TBF_R']-avg_singles_R, 3))
            doubles_R = max(0, round(2*pitcher['2B_R']/pitcher['TBF_R']-avg_doubles_R, 3))
            triples_R = max(0, round(2*pitcher['3B_R']/pitcher['TBF_R']-avg_triples_R, 3))
            home_runs_R = max(0, round(2*pitcher['HR_R']/pitcher['TBF_R']-avg_home_runs_R, 3))
            walks_R = max(0, round(2*pitcher['BB_R']/pitcher['TBF_R']-avg_walks_R, 3))
            so_R = max(0, round(2*pitcher['SO_R']/pitcher['TBF_R']-avg_so_R, 3))

            new_pitcher = Pitcher(self.city, pitcher['Name'], pitcher['Throws'], singles_L, doubles_L, triples_L, 
                                  home_runs_L, walks_L, so_L, singles_R, doubles_R, triples_R, 
                                  home_runs_R, walks_R, so_R, 0, 0)
            
            self.pitchers.append(new_pitcher)
            
            if(new_pitcher.name==starters['closer']):    # Assigning the team's projected closer to self.closer
                self.closer = new_pitcher

        def make_lineup(batters, starters, league):
            """Making a lineup of batters based on projected lineup
        
            Args:
                batters: dataframe of batters
                starters: dict with names in projected starting lineup
            Returns:
                lineup: list of Batter instances in order of projected lineup
            """
            lineup = []
            
            for starter in starters['lineup']:
                for batter in batters:
                    if(batter.name==starter):
                        lineup.append(batter)
                        
            if(league=='NL'):
                # If not all batters in projected lineup have data, create a batter to be in the lineup
                while(len(lineup)<8):
                    lineup.append(Batter(self.city, 'extra' +str(len(lineup)), 'R', .13, .04, .01, .02, 
                                                  .03, .25, .13, .04, .01, .02, .03, .25, .02, .01))
                # Adding batter to hit in pitcher's spot
                lineup.append(Batter(self.city, 'pitcher', 'R', .06, .01, .002, .005, .02, .5, .06, .01, 
                                              .002, .005, .02, .5, 0, 0))
            elif(league=='AL'):
                # If not all batters in projected lineup have data, create a batter to be in the lineup
                while(len(lineup)<9):
                    lineup.append(Batter(self.city, 'extra' +str(len(lineup)), 'R', .13, .04, .01, .02, 
                                                  .03, .25, .13, .04, .01, .02, .03, .25, .02, .01))             
                
            return lineup
        
        def make_rotation(pitchers, starters):
            """Making a rotation of starting pitchers based on projected rotation
        
            Args:
                pitchers: dataframe of pitchers
                starters: dict with names in projected starting lineup
            Returns:
                rotation: list of Pitcher instances in order of projected rotation
            """
            rotation = []
            
            for starter in starters['rotation']:
                for pitcher in pitchers:
                    if(pitcher.name==starter):
                        rotation.append(pitcher)
                        
            while(len(rotation)<5):
                # If not all pitchers in projected rotation have data, create a pitcher to be in the rotation
                rotation.append(Pitcher(self.city, 'extra'+ str(len(rotation)), 'R', .16, .06, .02, .03, 
                                                 .05, .2, .16, .06, .02, .03, .05, .2, 0, 0))
                
            return rotation
        
        def make_bullpen(pitchers, relievers_df):
            """Making a bullpen of relievers (except the closer) based on projected bullpen
        
            Args:
                pitchers: dataframe of pitchers
                relievers_df: df with names of projected bullpen members and their projected innings pitched
            Returns:
                bullpen_df: df of Pitcher instances and a value that will be used to probabilistically choose relievers
            """
            l_reliever_dict = {}
            l_innings_total = 0
            r_reliever_dict = {}
            r_innings_total = 0
            
            for index, reliever in relievers_df.iterrows():    # Finding relivers from pitcher_df
                for pitcher in pitchers:
                    if(reliever['Name']==pitcher.name and reliever['Name']!=starters['closer']):
                        if(pitcher.hand=='L'):
                            l_reliever_dict.update({pitcher: float(reliever['IP'])})
                            l_innings_total+=float(reliever['IP'])
                        elif(pitcher.hand=='R'):
                            r_reliever_dict.update({pitcher: float(reliever['IP'])})
                            r_innings_total+=float(reliever['IP'])
                        
            relief_df = pd.DataFrame(columns=['Pitcher', 'value'])
            l_value = 0
            for pitcher, ip in l_reliever_dict.items():    # Adding lefty relievers to relief_df with a value
                l_value+=(ip/l_innings_total)
                relief_df = relief_df.append({'Pitcher': pitcher, 'value': l_value}, ignore_index=True)

            r_value = 1
            for pitcher, ip in r_reliever_dict.items():# Adding righty relievers to relief_df with a value
                r_value+=(ip/r_innings_total)
                relief_df = relief_df.append({'Pitcher': pitcher, 'value': r_value}, ignore_index=True)
                      
            return relief_df
            
            
        self.lineup = make_lineup(self.batters, self.starters, self.league)
        self.rotation = make_rotation(self.pitchers, self.starters)
        self.bullpen_df = make_bullpen(self.pitchers, self.relievers_df)
        
    def choose_pitcher(self, outs, inning, runs, opp_runs, lineup, lineup_index):
        """Returns a pitcher to pitch for a team given game attributes
        
            Args:
                outs: current number of outs in inning
                runs: number of runs team has
                opp_runs: number of runs opposing team has
                lineup: opposing team's lineup
                lineup_index: place in lineup opposing team is
            Returns:
                instance of Pitcher class to pitch in game
            """

        self.available_pitchers = []
        for pitcher in self.pitchers:  # Adding pitchers who have not already pitched to a list
            if pitcher not in self.current_pitcher_list:
                self.available_pitchers.append(pitcher)
            
        # If Pitcher hasn't faced the minimum of 3 batters or finished a hlaf inning, they remain in game
        if(self.current_pitcher.current_bf<3 and self.current_pitcher.finished_half==False): 
            return self.current_pitcher
        
        # Creating variable total that uses game factors to determine if pitcher should get taken out
        total = ((self.current_pitcher.current_bf-self.current_pitcher.current_outs)*3 + 
                 self.current_pitcher.current_outs + self.current_pitcher.current_runs*3)

        if(self.current_pitcher.current_runs-self.current_pitcher.current_outs/3.0>0):
            total+=(self.current_pitcher.current_runs-self.current_pitcher.current_outs/3)*2

        if(self.current_pitcher.hand==lineup[lineup_index].hand):
            total-=3
            
        if(self.current_pitcher.finished_half):
            total+=2
        
        # If total value of pitcher is below some arbitrary threshold(50 for starters, 8 for relievers), leave them in
        if(total<=50 and self.current_pitcher in self.rotation or total<=8 and self.current_pitcher not 
           in self.rotation):
                
            return self.current_pitcher
        else:  # If current pitcher has total value greater than threshold, find new reliever
            
            if(inning==8 and outs>0 and self.closer not in self.current_pitcher_list 
               and runs-opp_runs>0 and runs-opp_runs<3): # Put closer in game if game is close and in late innings
                self.current_pitcher = self.closer
                self.current_pitcher_list.append(self.current_pitcher)
                return self.current_pitcher
            elif(inning>8 and self.closer not in self.current_pitcher_list and runs-opp_runs>0 and runs-opp_runs<4):
                self.current_pitcher = self.closer
                self.current_pitcher_list.append(self.current_pitcher)
                return self.current_pitcher
            else:
                # If game not close or game not in late innings, put non-closer reliever in game
                left_count = 0    # Number of lefties out of next three batters
                right_count = 0   # Number of righties out of next three batters

                for num in range(3):
                    if(lineup[(lineup_index+num)%9].hand=='L'):
                        left_count+=1
                    elif(lineup[(lineup_index+num)%9].hand=='R'):
                        right_count+=1

                if(left_count>right_count):  # If more of the next 3 batters are lefties, hand_indicator = 0
                    hand_indicator = 0
                else:                        # If more or equal of the next 3 batters are righties, hand_indicator = 1
                    hand_indicator = 1

                reliever_available = False   # False if no relievers are available
                lefties_available = 0
                righties_available = 0
                for index, row in self.bullpen_df.iterrows():  # Finding number of lefty and righty relievers available
                    if(row['Pitcher'] in self.available_pitchers):
                        reliever_available = True

                        if(row['Pitcher'].hand=='L'):
                            lefties_available+=1
                        elif(row['Pitcher'].hand=='R'):
                            righties_available+=1

                if(reliever_available):    # If a reliever is available, probabalistically putting one in
                    if(lefties_available>0 and hand_indicator==0 or righties_available>0 and hand_indicator==1):
                        attempts = 0

                        while(attempts<100):
                            num = random.uniform(0+hand_indicator, 1+hand_indicator)

                            for index, row in self.bullpen_df.iterrows():
                                if(num<row['value'] and row['Pitcher'] not in self.current_pitcher_list):
                                    self.current_pitcher = row['Pitcher']
                                    self.current_pitcher_list.append(self.current_pitcher)
                                    return self.current_pitcher

                            attempts+=1
                    hand_indicator = -(hand_indicator-1)      


                    if(lefties_available>0 or righties_available>0):
                        attempts = 0

                        while(attempts<100):
                            num = random.uniform(0+hand_indicator, 1+hand_indicator)

                            for index, row in self.bullpen_df.iterrows():
                                if(num<row['value'] and row['Pitcher'] not in self.current_pitcher_list):
                                    self.current_pitcher = row['Pitcher']
                                    self.current_pitcher_list.append(self.current_pitcher)
                                    return self.current_pitcher

                            attempts+=1
                    else:    # If no non-closer relievers available, put in closer or another starter in the rotation
                        if(self.closer in self.available_pitchers):
                            self.current_pitcher = self.closer
                            self.current_pitcher_list.append(self.current_pitcher)
                            return self.current_pitcher
                        else:
                            num = random.randint(0, len(self.available_pitchers)-1)
                            self.current_pitcher = self.available_pitchers[num]
                            self.current_pitcher_list.append(self.current_pitcher)
                            return self.current_pitcher
                                                                              
        return self.current_pitcher
                          
    def restart(self):
        """Resets values for position in lineup, runners on base, and runs if a game is to be started
        """
        for pitcher in self.current_pitcher_list:
            pitcher.current_bf = 0
            pitcher.current_outs = 0
            pitcher.current_runs = 0
            pitcher.finished_half = False
            
        self.current_pitcher_list = []   
        self.batting_index = 0
        self.current_pitcher = None
        self.starter_pitching = True
        self.runners = [None, None, None, 0]
        self.runs = 0
        
    def reset_runners(self):
        """Resets baserunners after each half inning
        """
        self.runners = [None, None, None, 0]
    
    def cum_stats_df(self):
        """Class that returns dataframes of stats for batters and pitchers
    
        Returns:
            batter_df: dataframe of batter stats
            pitcher_df: dataframe of pitcher stats
        """
        batter_df = pd.DataFrame(columns=['Name', 'PA', 'AB', 'H', '1B', '2B', '3B', 'HR', 'BB', 'K', 'AVG', 'OBP', 
                                          'SLG', 'OPS', 'R', 'RBI', 'SB', 'CS'])
        
        for b in self.batters:
            batter_values = [b.name, b.cum_pa_L+b.cum_pa_R, b.cum_pa_L+b.cum_pa_R-(b.cum_walks_L+b.cum_walks_R), 
                             b.hits(), b.cum_singles_L+b.cum_singles_R, b.cum_doubles_L+b.cum_doubles_R, 
                             b.cum_triples_L+b.cum_triples_R, b.cum_home_runs_L+b.cum_home_runs_R, 
                             b.cum_walks_L+b.cum_walks_R, b.strikeouts(), b.avg(), b.obp(), b.slg(), b.ops(), b.cum_runs, 
                             b.cum_rbi, b.cum_sb, b.cum_cs]
            batter_df = batter_df.append(pd.Series(batter_values, index=batter_df.columns), ignore_index=True)
        batter_df.sort_values(by=['PA'], ascending=False, inplace=True)
        
        pitcher_df = pd.DataFrame(columns=['Name', 'IP', 'K', 'H', 'BB', 'ERA', 'WHIP', 'HR','AVG', 'OBP', 
                                          'SLG', 'OPS', 'R', 'SB', 'CS'])
        
        for p in self.pitchers:
            pitcher_values = [p.name, p.ip(), p.strikeouts(), p.hits(), p.cum_walks_L+p.cum_walks_R, p.era(), p.whip(), 
                              p.cum_home_runs_L+p.cum_home_runs_R, p.avg(), p.obp(), p.slg(), p.ops(),
                              p.cum_runs, p.cum_sb, p.cum_cs]
            pitcher_df = pitcher_df.append(pd.Series(pitcher_values, index=pitcher_df.columns), ignore_index=True)
        pitcher_df.sort_values(by=['IP'], ascending=False, inplace=True) 
        
        return batter_df, pitcher_df
    
class Game:
    """Class that holds information for a Game.
    
    Attributes:

    """
    
    def __init__(self, away_team, home_team):
        """Initializes values for this class
        
        Args:
            away_team: away team name
            home_team: home team name
        """
        self.away_team = away_team
        self.home_team = home_team
                                
    def get_stolen_base_result(self, runners, pitcher, num):
        """stolen base result of a batter
        
        Args:
            runners: list of runners on base
            pitcher: pitcher currently pitching
            num: random number used to determine whether or not batter should steal base
        Returns:
            result of stolen base attempt (or None if no attempt)
        """
        if(runners[0]!=None and runners[1]==None):   # If first base occupied and second base not, steal possible
            if(num>runners[0].sb_rate):    # No stolen base attempt if random number greater than runner sb_rate
                return None
            else:    # Stolen base attempt
                x = random.uniform(0,1)
                if(x<runners[0].sb_success):  # Successful stolen base attempt
                    runners[0].cum_sb+=1
                    pitcher.cum_sb+=1
                    runners[1] = runners[0]
                    runners[0] = None
                    return None
                        
                else:    # Batter caught stealing
                    runners[0].cum_cs+=1
                    pitcher.cum_cs+=1
                    runners[0] = None
                    return "out"
        else:
            return None
        
    def get_result(self, batter, pitcher, outs, runners, num):
        """Using player attributes and a random number, a result is created for a plate appearance
        
        Args:
            batter: instance of batter class
            pitcher: instance of pitcher class
            outs: number of outs in inning
            runners: List of where runners are
            num: random number between 0 and 2
        Returns:
            str: refers to result based on random number (ie "walk")
        """      
        if(pitcher.hand=='L'):
            batter_against = 'L'
        elif(pitcher.hand=='R'):
            batter_against = 'R'
              
        if(batter.hand=='L'):
            pitcher_against = 'L'
        elif(batter.hand=='R'):
            pitcher_against = 'R'
        elif(batter.hand=='S'):
            if(pitcher.hand=='L'):
                pitcher_against='R'
            elif(pitcher.hand=='R'):
                pitcher_against='L'
        
        pitcher.current_bf+=1
    
        if(batter_against=='L' and pitcher_against=='L'):
            batter.cum_pa_L+=1
            pitcher.cum_pa_L+=1
            if(num<batter.singles_L):
                batter.cum_singles_L+=1
                pitcher.cum_singles_L+=1
                return "single"
            elif(num<batter.doubles_L):
                batter.cum_doubles_L+=1
                pitcher.cum_doubles_L+=1
                return "double"
            elif(num<batter.triples_L):
                batter.cum_triples_L+=1
                pitcher.cum_triples_L+=1
                return "triple"
            elif(num<batter.home_runs_L):
                batter.cum_home_runs_L+=1
                pitcher.cum_home_runs_L+=1
                return "home_run"
            elif(num<batter.walks_L):
                batter.cum_walks_L+=1
                pitcher.cum_walks_L+=1
                return "walk"
            elif(num<batter.so_L):
                batter.cum_so_L==1                
                pitcher.cum_outs_L+=1
                pitcher.cum_so_L+=1
                return "k"
            elif(num<1):
                if(outs<2 and runners[0]!=None and num>.9):
                    batter.cum_gdp+=1
                    pitcher.cum_outs_L+=2
                    pitcher.cum_gdp+=1
                    return "gdp"
                else:
                    pitcher.cum_outs_L+=1
                    return "out"
            elif(num<pitcher.singles_L+1):
                batter.cum_singles_L+=1
                pitcher.cum_singles_L+=1
                return "single"
            elif(num<pitcher.doubles_L+1):
                batter.cum_doubles_L+=1
                pitcher.cum_doubles_L+=1
                return "double"
            elif(num<pitcher.triples_L+1):
                batter.cum_triples_L+=1
                pitcher.cum_triples_L+=1
                return "triple"
            elif(num<pitcher.home_runs_L+1):
                batter.cum_home_runs_L+=1
                pitcher.cum_home_runs_L+=1
                return "home_run"
            elif(num<pitcher.walks_L+1):
                batter.cum_walks_L+=1
                pitcher.cum_walks_L+=1
                return "walk"
            elif(num<pitcher.so_L):
                batter.cum_so_L==1                
                pitcher.cum_outs_L+=1
                pitcher.cum_so_L+=1
                return "k"
            else:
                if(outs<2 and runners[0]!=None and num>1.9):
                    batter.cum_gdp+=1
                    pitcher.cum_outs_L+=2
                    pitcher.cum_gdp+=1
                    return "gdp"
                else:
                    pitcher.cum_outs_L+=1
                    return "out"
        elif(batter_against=='R' and pitcher_against=='L'):
            batter.cum_pa_R+=1
            pitcher.cum_pa_L+=1
            if(num<batter.singles_R):
                batter.cum_singles_R+=1
                pitcher.cum_singles_L+=1
                return "single"
            elif(num<batter.doubles_R):
                batter.cum_doubles_R+=1
                pitcher.cum_doubles_L+=1
                return "double"
            elif(num<batter.triples_R):
                batter.cum_triples_R+=1
                pitcher.cum_triples_L+=1
                return "triple"
            elif(num<batter.home_runs_R):
                batter.cum_home_runs_R+=1
                pitcher.cum_home_runs_L+=1
                return "home_run"
            elif(num<batter.walks_R):
                batter.cum_walks_R+=1
                pitcher.cum_walks_L+=1
                return "walk"
            elif(num<batter.so_R):
                batter.cum_so_R==1
                pitcher.cum_outs_L+=1
                pitcher.cum_so_L+=1
                return "k"
            elif(num<1):
                if(outs<2 and runners[0]!=None and num>.9):
                    batter.cum_gdp+=1
                    pitcher.cum_outs_L+=2
                    pitcher.cum_gdp+=1
                    return "gdp"
                else:
                    pitcher.cum_outs_L+=1
                    return "out"
            elif(num<pitcher.singles_L+1):
                batter.cum_singles_R+=1
                pitcher.cum_singles_L+=1
                return "single"
            elif(num<pitcher.doubles_L+1):
                batter.cum_doubles_R+=1
                pitcher.cum_doubles_L+=1
                return "double"
            elif(num<pitcher.triples_L+1):
                batter.cum_triples_R+=1
                pitcher.cum_triples_L+=1
                return "triple"
            elif(num<pitcher.home_runs_L+1):
                batter.cum_home_runs_R+=1
                pitcher.cum_home_runs_L+=1
                return "home_run"
            elif(num<pitcher.walks_L+1):
                batter.cum_walks_R+=1
                pitcher.cum_walks_L+=1
                return "walk"
            elif(num<pitcher.so_L):
                batter.cum_so_R==1
                pitcher.cum_outs_L+=1
                pitcher.cum_so_L+=1
                return "k"
            else:
                if(outs<2 and runners[0]!=None and num>1.9):
                    batter.cum_gdp+=1
                    pitcher.cum_outs_L+=2
                    pitcher.cum_gdp+=1
                    return "gdp"
                else:
                    pitcher.cum_outs_L+=1
                    return "out"
        elif(batter_against=='L' and pitcher_against=='R'):
            batter.cum_pa_L+=1
            pitcher.cum_pa_R+=1
            if(num<batter.singles_L):
                batter.cum_singles_L+=1
                pitcher.cum_singles_R+=1
                return "single"
            elif(num<batter.doubles_L):
                batter.cum_doubles_L+=1
                pitcher.cum_doubles_R+=1
                return "double"
            elif(num<batter.triples_L):
                batter.cum_triples_L+=1
                pitcher.cum_triples_R+=1
                return "triple"
            elif(num<batter.home_runs_L):
                batter.cum_home_runs_L+=1
                pitcher.cum_home_runs_R+=1
                return "home_run"
            elif(num<batter.walks_L):
                batter.cum_walks_L+=1
                pitcher.cum_walks_R+=1
                return "walk"
            elif(num<batter.so_L):
                batter.cum_so_L==1
                pitcher.cum_outs_R+=1
                pitcher.cum_so_R+=1
                return "k"
            elif(num<1):
                if(outs<2 and runners[0]!=None and num>.9):
                    batter.cum_gdp+=1
                    pitcher.cum_outs_R+=2
                    pitcher.cum_gdp+=1
                    return "gdp"
                else:
                    pitcher.cum_outs_R+=1
                    return "out"
            elif(num<pitcher.singles_R+1):
                batter.cum_singles_L+=1
                pitcher.cum_singles_R+=1
                return "single"
            elif(num<pitcher.doubles_R+1):
                batter.cum_doubles_L+=1
                pitcher.cum_doubles_R+=1
                return "double"
            elif(num<pitcher.triples_R+1):
                batter.cum_triples_L+=1
                pitcher.cum_triples_R+=1
                return "triple"
            elif(num<pitcher.home_runs_R+1):
                batter.cum_home_runs_L+=1
                pitcher.cum_home_runs_R+=1
                return "home_run"
            elif(num<pitcher.walks_R+1):
                batter.cum_walks_L+=1
                pitcher.cum_walks_R+=1
                return "walk"
            elif(num<pitcher.so_R):
                batter.cum_so_L==1
                pitcher.cum_outs_R+=1
                pitcher.cum_so_R+=1
                return "k"
            else:
                 if(outs<2 and runners[0]!=None and num>1.9):
                    batter.cum_gdp+=1
                    pitcher.cum_outs_R+=2
                    pitcher.cum_gdp+=1
                    return "gdp"
                 else:
                    pitcher.cum_outs_R+=1
                    return "out"
        elif(batter_against=='R' and pitcher_against=='R'):
            batter.cum_pa_R+=1
            pitcher.cum_pa_R+=1
            if(num<batter.singles_R):
                batter.cum_singles_R+=1
                pitcher.cum_singles_R+=1
                return "single"
            elif(num<batter.doubles_R):
                batter.cum_doubles_R+=1
                pitcher.cum_doubles_R+=1
                return "double"
            elif(num<batter.triples_R):
                batter.cum_triples_R+=1
                pitcher.cum_triples_R+=1
                return "triple"
            elif(num<batter.home_runs_R):
                batter.cum_home_runs_R+=1
                pitcher.cum_home_runs_R+=1
                return "home_run"
            elif(num<batter.walks_R):
                batter.cum_walks_R+=1
                pitcher.cum_walks_R+=1
                return "walk"
            elif(num<batter.so_R):
                batter.cum_so_R==1
                pitcher.cum_outs_R+=1
                pitcher.cum_so_R+=1
                return "k"
            elif(num<1):
                 if(outs<2 and runners[0]!=None and num>.9):
                    batter.cum_gdp+=1
                    pitcher.cum_outs_R+=2
                    pitcher.cum_gdp+=1
                    return "gdp"
                 else:
                    pitcher.cum_outs_R+=1
                    return "out"
            elif(num<pitcher.singles_R+1):
                batter.cum_singles_R+=1
                pitcher.cum_singles_R+=1
                return "single"
            elif(num<pitcher.doubles_R+1):
                batter.cum_doubles_R+=1
                pitcher.cum_doubles_R+=1
                return "double"
            elif(num<pitcher.triples_R+1):
                batter.cum_triples_R+=1
                pitcher.cum_triples_R+=1
                return "triple"
            elif(num<pitcher.home_runs_R+1):
                batter.cum_home_runs_R+=1
                pitcher.cum_home_runs_R+=1
                return "home_run"
            elif(num<pitcher.walks_R+1):
                batter.cum_walks_R+=1
                pitcher.cum_walks_R+=1
                return "walk"
            elif(num<pitcher.so_R):
                batter.cum_so_R==1
                pitcher.cum_outs_R+=1
                pitcher.cum_so_R+=1
                return "k"
            else:
                if(outs<2 and runners[0]!=None and num>1.9):
                    batter.cum_gdp+=1
                    pitcher.cum_outs_R+=2
                    pitcher.cum_gdp+=1
                    return "gdp"
                else:
                    pitcher.cum_outs_R+=1
                    return "out"      
            
    def move_runners(self, team, pitcher, result):
        """Based on the result of a batter's plate appearance, runners are moved and runs may be added to team's total 
        
        Args:
            team: instance team that the batter was on
            pitcher: pitcher pitching to team
            result: str of result for batter (ie "walk")
        """
        if(result=='single'):
            if(team.runners[2]!=None):
                team.runners[2].cum_runs+=1
                pitcher.cum_runs+=1
                pitcher.current_runs+=1
                team.lineup[team.batting_index].cum_rbi+=1
                team.runners[3]+=1
                
            team.runners[2] = team.runners[1]
            team.runners[1] = team.runners[0]
            team.runners[0] = team.lineup[team.batting_index]

        elif(result=='double'):
            if(team.runners[2]!=None):
                team.runners[2].cum_runs+=1
                pitcher.cum_runs+=1
                pitcher.current_runs+=1
                team.lineup[team.batting_index].cum_rbi+=1
                team.runners[3]+=1
            if(team.runners[1]!=None):
                team.runners[1].cum_runs+=1
                pitcher.cum_runs+=1
                pitcher.current_runs+=1
                team.lineup[team.batting_index].cum_rbi+=1
                team.runners[3]+=1
                
            team.runners[2] = team.runners[0]
            team.runners[1] = team.lineup[team.batting_index]
            team.runners[0] = None

        elif(result=='triple'):
            if(team.runners[2]!=None):
                team.runners[2].cum_runs+=1
                pitcher.cum_runs+=1
                pitcher.current_runs+=1
                team.lineup[team.batting_index].cum_rbi+=1
                team.runners[3]+=1
            if(team.runners[1]!=None):
                team.runners[1].cum_runs+=1
                pitcher.cum_runs+=1
                pitcher.current_runs+=1
                team.lineup[team.batting_index].cum_rbi+=1
                team.runners[3]+=1
            if(team.runners[0]!=None):
                team.runners[0].cum_runs+=1
                pitcher.cum_runs+=1
                pitcher.current_runs+=1
                team.lineup[team.batting_index].cum_rbi+=1
                team.runners[3]+=1
                
            team.runners[2] = team.lineup[team.batting_index]
            team.runners[1] = None
            team.runners[0] = None

        elif(result=='home_run'):
            team.lineup[team.batting_index].cum_runs+=1
            pitcher.cum_runs+=1
            pitcher.current_runs+=1
            team.lineup[team.batting_index].cum_rbi+=1
            team.runners[3]+=1
            
            if(team.runners[2]!=None):
                team.runners[2].cum_runs+=1
                pitcher.cum_runs+=1
                pitcher.current_runs+=1
                team.lineup[team.batting_index].cum_rbi+=1
                team.runners[3]+=1
            if(team.runners[1]!=None):
                team.runners[1].cum_runs+=1
                pitcher.cum_runs+=1
                pitcher.current_runs+=1
                team.lineup[team.batting_index].cum_rbi+=1
                team.runners[3]+=1
            if(team.runners[0]!=None):
                team.runners[0].cum_runs+=1
                pitcher.cum_runs+=1
                pitcher.current_runs+=1
                team.lineup[team.batting_index].cum_rbi+=1
                team.runners[3]+=1
                
            team.runners[2] = None
            team.runners[1] = None
            team.runners[0] = None

        elif(result=='walk'):
            temp_2 = team.runners[2]
            temp_1 = team.runners[1]
            temp_0 = team.runners[0]
            
            team.runners[0] = team.lineup[team.batting_index]
            
            if(temp_0!=None):
                team.runners[1] = temp_0
                
                if(temp_1!=None):
                    team.runners[2] = temp_1
                    
                    if(temp_2!=None):
                        temp_2.cum_runs+=1
                        pitcher.cum_runs+=1
                        pitcher.current_runs+=1
                        team.lineup[team.batting_index].cum_rbi+=1
                        team.runners[3]+=1
                        
        elif(result=="gdp"):
            temp_2 = team.runners[2]
            temp_1 = team.runners[1]
            temp_0 = team.runners[0]
            rand_num = random.uniform(0,1)
            
            if(temp_1==None and temp_2==None):    # If only runner on first
                team.runners[0]==None
            elif(temp_1!=None and temp_2==None):    # If runners on first and second
                if(rand_num<.02):   # Outs made at first and third
                    team.runners[1] = temp_0
                    team.runners[0] = None
                elif(rand_num<.08):   # Outs made at third and second
                    team.runners[1] = None
                    team.runners[0] = team.lineup[team.batting_index]
                else:
                    team.runners[2] = temp_1
                    team.runners[1] = None
                    team.runners[0] = None
            elif(temp_1==None and temp_2!=None):    # If runners on first and third
                if(rand_num<.01):   # Batter goes to first
                    team.runners[0] = team.lineup[team.batting_index]
                    team.runners[2] = None
                elif(rand_num<.02):   # Batter goes to second
                    team.runners[0] = None
                    team.runners[1] = team.lineup[team.batting_index]
                    team.runners[2] = None
                else: # Runner scores, bases empty
                    temp_2.cum_runs+=1
                    pitcher.cum_runs+=1
                    pitcher.current_runs+=1
                    team.runners[3]+=1
                    team.runners[2] = None
                    team.runners[1] = None
                    team.runners[0] = None
            elif(temp_1!=None and temp_2!=None):    # If bases loaded
                if(rand_num<.6):   # Outs made at second and first
                    temp_2.cum_runs+=1
                    pitcher.cum_runs+=1
                    pitcher.current_runs+=1
                    team.runners[3]+=1
                    team.runners[0] = None
                    team.runners[1] = None
                    team.runners[2] = temp_1
                elif(rand_num<.9):   # Outs made at home and first
                    team.runners[1] = temp_0
                    team.runners[2] = temp_1
                    team.runners[0] = None
                else:    # Outs made at third and home
                    team.runners[2] = None
                    team.runners[1] =temp_0
                    team.runners[0] = team.lineup[team.batting_index]

        if(team.runners[3]>0):
            team.runs+=team.runners[3]
            team.runners[3] = 0  
            
    def print_result(self):
        """Printed score of a game
        """ 
        print("final score: " + self.away_team.city + " " + str(self.away_team.runs) + " " 
              + self.home_team.city + " " + str(self.home_team.runs))     
    
    def print_record(self):
        """Printed record of teams
        """ 
        print(str(self.away_team.wins) + "-" + str(self.away_team.losses))
        print(str(self.home_team.wins) + "-" + str(self.home_team.losses))     
   
    def play(self):
        """A game between two instances of the team class is played
        """ 
        self.away_team.restart()
        self.away_team.game_no+=1
        away_team_rotation_index = (self.away_team.game_no-1)%5
        self.home_team.restart() 
        self.home_team.game_no+=1
        home_team_rotation_index = (self.home_team.game_no-1)%5
        inning = 1
        game_over = False
        
        while(inning<10 or self.away_team.runs==self.home_team.runs):
            for side in [self.away_team, self.home_team]:
                if(inning==9 and side==self.home_team and self.home_team.runs>self.away_team.runs):
                    game_over = True
                    break
                    
                outs = 0
                
                if(inning==1):
                    # Determining starting pitchers
                    if(side==self.away_team):
                        pitcher = self.home_team.rotation[home_team_rotation_index]
                        self.home_team.current_pitcher_list.append(pitcher)
                        self.home_team.current_pitcher = pitcher
                    elif(side==self.home_team):
                        pitcher = self.away_team.rotation[away_team_rotation_index]
                        self.away_team.current_pitcher_list.append(pitcher)
                        self.away_team.current_pitcher = pitcher

                while(outs<3):
                    # Choosing a pitcher for a team based on game factors
                    if(side==self.away_team):
                        pitcher = self.home_team.choose_pitcher(outs, inning, side.runs, self.home_team.runs, 
                                                                side.lineup, side.batting_index)
                    elif(side==self.home_team):
                        pitcher = self.away_team.choose_pitcher(outs, inning, side.runs, self.away_team.runs, 
                                                                side.lineup, side.batting_index)
                    
                    # Determining whether stolen base should be attempted
                    stolen_base_num = random.uniform(0,1)
                    stolen_base_result = self.get_stolen_base_result(side.runners, pitcher, stolen_base_num)
                    
                    if(stolen_base_result=="out"):   # If failed stolen base, out is recorded
                        pitcher.current_outs+=1
                        outs+=1
                                
                        if(outs>2):   # If outs = 3, end of inning
                            pitcher.finished_half = True
                            break
                    
                    # Determining outcome of at bat
                    result_num = random.uniform(0,2)
                    result = self.get_result(side.lineup[side.batting_index], pitcher, outs, side.runners, result_num)
                        
                    if(result=="out" or result=='k' or result=='sf'):    # Batter made an out
                        pitcher.current_outs+=1
                        outs+=1
                        
                        if(outs>2):   # If outs = 3, end of inning
                            pitcher.finished_half = True
                            side.batting_index+=1    # Gettting next batter in lineup
                            side.batting_index = side.batting_index%9
                            break
                            
                    elif(result=="gdp"):    # Batter made an out
                        pitcher.current_outs+=2
                        outs+=2
                        
                        if(outs>2):   # If outs = 3, end of inning
                            pitcher.finished_half = True
                            side.batting_index+=1    # Gettting next batter in lineup
                            side.batting_index = side.batting_index%9
                            break
                        else:
                            self.move_runners(side, pitcher, result)
                            
                    else:    # Batter reached, runners are moved
                        self.move_runners(side, pitcher, result)
                        if(inning>9 and self.away_team.runs<self.home_team.runs):
                            game_over = True
                    
                    if(game_over):    # If game is over, escape game loop
                        break
                        
                    side.batting_index+=1    # Gettting next batter in lineup
                    side.batting_index = side.batting_index%9
                    
                if(game_over):    # If game is over, escape game loop
                    break
                        
                side.reset_runners()    # Runners are reset after half inning
            

            if(game_over):    # If game is over, escape game loop
                break   
                
            inning+=1
        
        # After game, updating team records and number of cumulative runs
        if(self.away_team.runs>self.home_team.runs):
            self.away_team.wins+=1
            self.home_team.losses+=1
        elif(self.away_team.runs<self.home_team.runs):
            self.away_team.losses+=1
            self.home_team.wins+=1
        
        self.away_team.cum_runs+=self.away_team.runs
        self.away_team.cum_runs_allowed+=self.home_team.runs
        self.home_team.cum_runs+=self.home_team.runs
        self.home_team.cum_runs_allowed+=self.away_team.runs             
        
class Season(Game):
    """Class that holds information for a Season of games.
    
    Attributes:

    """
    
    def __init__(self, away_team, home_team):
        """Initializes values for this class
        
        Args:
            away_team: away team name
            home_team: home team name
        """
        self.away_team = away_team
        self.home_team = home_team

    def simulate_season(self, games):
        """A season of games is simulated
        
        Args:
            games: number of games to be played
        """
        for game in range(1, games+1):
            g = Game(self.away_team, self.home_team)

            g.play() 
            
    def reset_season(self, teams):
        """Reinitializing teams
        
        Args:
            teams, Team instances to reinitialize
        """
        for team in teams:
            team.__init__(team.city, team.league, team.batter_df, team.pitcher_df, team.starters, team.relievers_df)
 
class FullSeason(Game):
    """Class that holds information for a Season of games
    
    Attributes:

    """
    
    def __init__(self, schedule):
        """Initializes values for this class
        
        Args:
            schedule: dataframe of season schedule
        """
        self.schedule = schedule

    def simulate_season(self):
        """A season of games is simulated
        """
        for date in self.schedule['date'].unique():
            for acronym, city in acronym_to_city_dict.items():
                away_team = team_dict_df[acronym_to_city_dict[acronym]]
                home_team_acronym = self.schedule.loc[self.schedule['date']==date][acronym].values[0]

                if(not pd.isna(home_team_acronym)):
                    home_team = team_dict_df[acronym_to_city_dict[home_team_acronym]]
                    g = Game(away_team, home_team)
                    g.play() 
  
    def reset_season(self):
        """Reinitializing teams
        """
        for city_name, team in team_dict_df.items():
            team.__init__(team.city, team.league, team.batter_df, team.pitcher_df, team.starters, team.relievers_df)   
            
    def print_record(self):
        """Printing records
        """
        for city_name, team in team_dict_df.items():
            print(team.city + ': ' + str(team.wins) + "-" + str(team.losses))
                       

In [221]:
team_dict_df = create_teams(batter_stats_df, pitcher_stats_df, starters_dict, relievers_dict)
g = Game(team_dict_df['Los Angeles Angels'], team_dict_df['Houston'])
g.play()
g.print_result()

inning 1
inning 2
inning 3
inning 4
inning 5
inning 6
inning 7
inning 8
inning 9
final score: Los Angeles Angels 2 Houston 3


In [227]:
a, b = team_dict_df['Los Angeles Dodgers'].cum_stats_df()
a

,Name,PA,AB,H,1B,2B,3B,HR,BB,K,AVG,OBP,SLG,OPS,R,RBI,SB,CS
9,Mookie Betts,785,667,195,123,39,5,28,118,0,0.292,0.399,0.492,0.891,125,61,26,3
4,Max Muncy,769,663,162,75,32,6,49,106,0,0.244,0.349,0.532,0.881,114,105,3,1
7,Justin Turner,746,676,222,146,36,3,37,70,0,0.328,0.391,0.555,0.946,103,80,0,5
8,Cody Bellinger,729,622,225,106,56,9,54,107,0,0.362,0.455,0.741,1.196,125,140,10,8
12,A.J. Pollock,714,657,154,87,35,2,30,57,0,0.234,0.296,0.431,0.727,75,91,4,1
6,Corey Seager,700,641,160,82,51,4,23,59,0,0.25,0.313,0.449,0.762,64,89,0,0
1,Will Smith,682,609,155,59,28,3,65,73,0,0.255,0.334,0.631,0.965,95,128,0,7
3,Gavin Lux,663,616,165,120,27,9,9,47,0,0.268,0.32,0.385,0.705,48,47,18,2
0,Austin Barnes,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Matt Beaty,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [228]:
print(team_dict_df['Houston'].rotation[0].singles_L)
print(team_dict_df['Houston'].rotation[0].doubles_L)
print(team_dict_df['Houston'].rotation[0].triples_L)
print(team_dict_df['Houston'].rotation[0].home_runs_L)
print(team_dict_df['Houston'].rotation[0].walks_L)
print(team_dict_df['Houston'].rotation[0].so_L)

print(team_dict_df['Houston'].rotation[0].singles_R)
print(team_dict_df['Houston'].rotation[0].doubles_R)
print(team_dict_df['Houston'].rotation[0].triples_R)
print(team_dict_df['Houston'].rotation[0].home_runs_R)
print(team_dict_df['Houston'].rotation[0].walks_R)
print(team_dict_df['Houston'].rotation[0].so_R)

0.007
0.028999999999999998
0.047
0.091
0.121
0.682
0.03
0.057999999999999996
0.073
0.131
0.138
4


In [230]:
print(team_dict_df['Houston'].lineup[0].singles_L)
print(team_dict_df['Houston'].lineup[0].doubles_L)
print(team_dict_df['Houston'].lineup[0].triples_L)
print(team_dict_df['Houston'].lineup[0].home_runs_L)
print(team_dict_df['Houston'].lineup[0].walks_L)
print(team_dict_df['Houston'].lineup[0].so_L)

print(team_dict_df['Los Angeles Angels'].lineup[2].singles_R)
print(team_dict_df['Los Angeles Angels'].lineup[2].doubles_R)
print(team_dict_df['Los Angeles Angels'].lineup[2].triples_R)
print(team_dict_df['Los Angeles Angels'].lineup[2].home_runs_R)
print(team_dict_df['Los Angeles Angels'].lineup[2].walks_R)
print(team_dict_df['Los Angeles Angels'].lineup[2].so_R)

6
0.149
0.162
0.235
0.46399999999999997
0.6699999999999999
0.168
0.226
0.241
0.301
0.368
0.618


In [95]:
print(team_dict_df['Houston'].lineup[3].hits())

3


In [174]:

g.play()
g.print_result()

final score: Los Angeles Angels 17 Houston 15


In [180]:
def convert_int(row, column):
    
    if(column in row.index):
         if(row[column]!=''):
            return(int(row[column]))
         else:
            return(0)
        
def convert_float(row, column):
    
    if(column in row.index):
        if(row[column]!=''):
            return(float(row[column]))
        else:
            return(0.0)
        
def reading_data(file_name):
    """Loads spreadsheet
    Args:
        file_name: path of file
    Returns:
        dataframe of data
    """
    return(pd.read_csv(file_name))
  
def writing_data(df, filename):
    """Writing to spreadsheet
    Args:
        df: data
        file_name: path of file
    """
    df.to_csv(filename, index=False)
    
def column_type(batter_stats, pitcher_stats):
    string_columns = ['Season', 'Name', 'Bats', 'Tm']
    int_columns = ['playerId', 'G_R', 'PA_R', 'AB_R',
                   'H_R', '1B_R', '2B_R', '3B_R', 'HR_R', 'R_R', 'RBI_R', 'BB_R', 'IBB_R','SO_R', 'HBP_R', 'SF_R',
                   'SH_R', 'GDP_R', 'SB_R', 'CS_R', 'G_L', 'PA_L', 'AB_L', 'H_L', '1B_L', '2B_L', '3B_L', 'HR_L', 
                   'R_L', 'RBI_L', 'BB_L', 'IBB_L', 'SO_L', 'HBP_L', 'SF_L', 'SH_L', 'GDP_L', 'SB_L', 'CS_L']
    float_columns = ['AVG_R', 'AVG_L']

    for column in batter_stats.columns:
        if(column in int_columns):
            batter_stats[column] = batter_stats.apply(lambda row: convert_int(row, column), axis=1)
        elif(column in float_columns):
            batter_stats[column] = batter_stats.apply(lambda row: convert_float(row, column), axis=1)

    string_columns = ['Season', 'Name', 'Throws', 'Team']
    int_columns = ['playerId', 'IP_L', 'TBF_L', 'H_L', '1B_L', '2B_L', '3B_L', 'R_L', 'ER_L', 'HR_L', 'BB_L', 'IBB_L', 
                   'HBP_L','SO_L', 'IP_R', 'TBF_R', 'H_R', '2B_R', '3B_R', 'R_R', 'ER_R', 'HR_R', 'BB_R', 'IBB_R', 
                   'HBP_R', 'SO_R']
    float_columns = ['ERA_L', 'AVG_L', 'OBP_L', 'SLG_L', 'wOBA_L', 'ERA_R', 'AVG_R', 'OBP_R', 'SLG_R', 'wOBA_R']


    for column in pitcher_stats.columns:
        if(column in int_columns):
            pitcher_stats[column] = pitcher_stats.apply(lambda row: convert_int(row, column), axis=1)
        elif(column in float_columns):
            pitcher_stats[column] = pitcher_stats.apply(lambda row: convert_float(row, column), axis=1)     
            
    return batter_stats, pitcher_stats
    
def date_formatter(row):
    """Formats dates
    Args:
        row: row of data
    Returns:
        date in proper form('mm/dd/yyyy')
    """
    if row.date == '':
        return(row.date)
    else:
        split_date = row.date.split('-')
        day = split_date[0]
        day = day.zfill(2)
        month = strptime(split_date[1],'%b').tm_mon
        date = str(month) +  '/' + day + '/' + '2020'
        date = datetime.datetime.strptime(date, '%m/%d/%Y')
        date = date.strftime('%m/%d/%Y')
        
    return date

def split_stats():   
    url = 'http://www.espn.com/mlb/player/splits/_/id/31283/type/batting3'

    headers = {'User-Agent': header_name}

    platoon_dict = {}
    data_columns = ['Split', 'Year', 'Tm', 'team_ID', 'Pos', 'POS', 'pos_season', 'Name', 'player', 'Age', 'age', 'G', 'PA', 'AB', 
                    'R', 'H', '2B', '3B', 'HR', 'SB', 'CS', 'BB', 'SO', 'HBP']



    source = requests.get(url, headers=headers)
    soup = BeautifulSoup(source.content, 'html.parser')
    left_columns = ['Name', 'Bats']
    right_columns = []
    name = soup.h1.get_text()

    for bio in soup.findAll('div', attrs={'class': 'player-bio'}):
        for info in bio.findAll('ul', attrs={'class': 'general-info'}):
            bats = info.get_text().split('Bats')[1][2]

    for table in soup.findAll('table', attrs={'class': 'tablehead'}):
        labels = table.find('tr', attrs={'class': 'colhead'})

        for header_value in labels.findAll('td'):
            left_columns.append(header_value.get_text()+'L')
            right_columns.append(header_value.get_text()+'R')

        df = pd.DataFrame(columns=(left_columns+right_columns))            
        player_values = [name, bats]

        for column_value in table.findAll('tr', attrs={'class': ['oddrow', 'evenrow']}):
            if(column_value.td.get_text() in ['vs. Left', 'vs. Right']):
                for value in column_value.findAll('td'):
                    player_values.append(value.get_text())

        df = df.append(pd.Series(player_values, index=df.columns), ignore_index=True)

    df.drop(['OverallL', 'OverallR'], axis=1, inplace=True)   
    print(df)

#df['date'] = df.apply(lambda row: date_formatter(row), axis=1)
#print(df)
    
#writing_data(batter_stats_df, 'mlb_batter_stats_2019.csv')
#writing_data(pitcher_stats_df, 'mlb_pitcher_stats_2019.csv')        

In [181]:
def batter_stats():

    url = 'http://www.espn.com/mlb/history/leaders/_/breakdown/season/year/2019/start/'
    links = [url+str(i) for i in range(1, 350, 50)]
    headers = {'User-Agent': header_name}

    batter_stats = pd.DataFrame()

    for (num, link) in enumerate(links):
        source = requests.get(link, headers=headers)
        soup = BeautifulSoup(source.content, 'html.parser')
        table = soup.find('table', attrs={'class': 'tablehead'})
        if(num==0):
            column_headers = []
            table_headers = table.find('tr', attrs={'class': 'colhead'})
            for header_value in table_headers.findAll('td'):
                column_headers.append(header_value.get_text())

            column_headers[0] = 'ID'
            batter_stats = pd.DataFrame(columns=column_headers)

        df = pd.DataFrame(columns=column_headers)            


        for player in table.find_all('tr', attrs={'class': re.compile('row player-10-')}):
            player_values = []
            player_values.append(player['class'][1].split('-')[2])

            values =  player.findAll('td')
            values.pop(0)
            for value in values:
                player_values.append(value.get_text())

            df = df.append(pd.Series(player_values, index=df.columns), ignore_index=True)

        batter_stats = pd.concat([batter_stats, df], ignore_index=True)

    return batter_stats

def pitcher_stats():
    url = 'http://www.espn.com/mlb/history/leaders/_/type/pitching/breakdown/season/year/2019/sort/wins/start/'
    links = [url+str(i) for i in range(1, 500, 50)]
    headers = {'User-Agent': header_name}

    pitcher_stats = pd.DataFrame()

    for (num, link) in enumerate(links):
        source = requests.get(link, headers=headers)
        soup = BeautifulSoup(source.content, 'html.parser')
        table = soup.find('table', attrs={'class': 'tablehead'})
        if(num==0):
            column_headers = []
            table_headers = table.find('tr', attrs={'class': 'colhead'})
            for header_value in table_headers.findAll('td'):
                column_headers.append(header_value.get_text())

            column_headers[0] = 'ID'
            pitcher_stats = pd.DataFrame(columns=column_headers)

        df = pd.DataFrame(columns=column_headers)            


        for player in table.find_all('tr', attrs={'class': re.compile('row player-10-')}):
            player_values = []
            player_values.append(player['class'][1].split('-')[2])

            values =  player.findAll('td')
            values.pop(0)
            for value in values:
                player_values.append(value.get_text())

            df = df.append(pd.Series(player_values, index=df.columns), ignore_index=True)

        pitcher_stats = pd.concat([pitcher_stats, df], ignore_index=True)

    return pitcher_stats

def projected_starters():

    url = 'https://www.mlb.com/news/projecting-every-mlb-lineup-rotation'
    headers = {'User-Agent': header_name}
    source = requests.get(url, headers=headers)
    soup = BeautifulSoup(source.content, 'html.parser')

    starters = {}

    for body in soup.findAll('div', attrs={'class': 'article-item__body'}):
        pitchers = []
        batters = []
        closer = None

        for team in body.findAll('p'):
            if(team.strong and team.strong.get_text().lower() in nickname_to_city_dict.keys()):
                current_team = team.strong.get_text().lower()
                a = team.get_text().split(')')
                b = []
                c = []
                d = []

                for entry in a:
                    b.append(str(entry.split('/')[0]))
                for entry in b:
                    c.append(str(entry.split(',')[0]))
                for entry in c:
                    d.append(entry.split(' or ')[0])
            
                for entry in d[1:]:
                    batters.append(unidecode.unidecode(entry.lstrip()))
                
            elif(team.strong and team.strong.get_text()=='Pitchers'):
                a = team.get_text().split(')')
                b = []
                c = []
                d = []

                for entry in a:
                    b.append(str(entry.split('/')[0]))
                for entry in b:
                    c.append(str(entry.split(',')[0]))
                for entry in c:
                    d.append(entry.split(' or ')[0])
            
                for entry in d[1:]:
                    pitchers.append(unidecode.unidecode(entry.lstrip()))
            elif(team.strong and team.strong.get_text()=='Closer:'):
                closer = unidecode.unidecode(team.find(['forge-entity', 'a']).get_text())
            else:
                continue

            if(closer!=None):
                starters.update({nickname_to_city_dict[current_team]: 
                                {'lineup': batters, 'rotation': pitchers, 'closer': closer}})
                pitchers = []
                batters = []
                closer = None
                
    return starters

def projected_relievers():
    league_relief_dict = {}
    url_list = []
    url_start = 'https://www.fangraphs.com/depthcharts.aspx?position=ALL&teamid='

    for num in range(1,31):
        url_list.append(url_start+str(num))

    for url in url_list:
        headers = {'User-Agent': header_name}
        source = requests.get(url, headers=headers)
        soup = BeautifulSoup(source.content, 'html.parser')
        relief_df = pd.DataFrame(columns=['Name', 'IP'])


        for a_tag in soup.findAll('a', attrs={'href': '#RP'}):
            team_name = a_tag.find_next('span').get_text()
            team_name = team_name.replace('\xa0', '')
            relief_table = a_tag.find_next('table')

            for row in relief_table.findAll('tr', attrs={'class': 'depth_reg'}):
                name_tag = row.find_next('td')
                name = name_tag.get_text()
                ip = name_tag.find_next('td').get_text()
                relief_df = relief_df.append({'Name': name, 'IP': ip}, ignore_index=True)

        league_relief_dict.update({nickname_to_city_dict[str(team_name.lower())]: relief_df})
        
    return league_relief_dict

def create_teams(batter_stats, pitcher_stats, starters, relievers):
    url_dict = {}
    url_start = 'https://www.espn.com/mlb/team/roster/_/name/'
    team_dict = {}
    

    for acronym, city in acronym_to_city_dict.items():
        url_dict.update({url_start+acronym.lower(): city})

    for url, city in url_dict.items():
        headers = {'User-Agent': header_name}
        source = requests.get(url, headers=headers)
        soup = BeautifulSoup(source.content, 'html.parser')
        player_id_dict = {}
        pitcher_df = pd.DataFrame()
        batter_df = pd.DataFrame()

        for section in soup.findAll('section'):
            for player in section.findAll('td', attrs={'class': 'Table__TD'}):
                row = player.find('a', attrs={'class': 'AnchorLink'})
                pos = player.find('span')

                if(row and row.get_text()!=''):
                    if(section.div.get_text()=='Pitchers'):
                        player_id_dict.update({row.get_text(): {int(row['href'].split('/')[-1]): 'P'}})
                    else:
                        player_id_dict.update({row.get_text(): {int(row['href'].split('/')[-1]): 'H'}})
                   
        for name, df in player_id_dict.items():
            for ID, pos in df.items():
                if(pos=='P'):
                    if(name in pitcher_stats['Name'].tolist()):
                        stats = pitcher_stats.loc[pitcher_stats['Name']==name]
                        col_headers = stats.columns
                    else:
                        stats = pd.Series({'Season': 2019, 'Name': name, 'playerId': 1234, 'Team': city, 'Throws': 'R', 
                                           'IP_L': 30, 'TBF_L': 130, 'ERA_L': 5.40, 'H_L': 30, '1B_L': 20, '2B_L': 5, 
                                           '3B_L': 1, 'R_L': 18, 'ER_L': 18, 'HR_L': 4, 'BB_L': 17, 'IBB_L': 0,  
                                           'HBP_L': 3, 'SO_L': 50, 'AVG_L': .250, 'OBP_L': .385, 'SLG_L': .544, 
                                           'wOBA_L': .370, 'IP_R': 30, 'TBF_R': 130, 'ERA_R': 5.40, 'H_R': 30, 
                                           '1B_R': 20, '2B_R': 5, '3B_R': 1, 'R_R': 18, 'ER_R': 18, 'HR_R': 4, 
                                           'BB_R': 17, 'IBB_R': 0, 'HBP_R': 3, 'SO_R': 50, 'AVG_R': .250, 
                                           'OBP_R': .385, 'SLG_R': .544, 'wOBA_R': .370})
                        
                        col_headers = stats.index
                    if(pitcher_df.empty):
                        pitcher_df = pd.DataFrame(columns=col_headers)
                    
                    pitcher_df = pitcher_df.append(stats, ignore_index=True)

                elif(pos=='H'):
                    if(name in batter_stats['Name'].tolist()):
                        stats = batter_stats.loc[batter_stats['Name']==name]
                        col_headers = stats.columns
                    else:
                        stats = pd.Series({'Season': 2019, 'Name': name, 'playerId': 1234, 'Team': city, 'Bats': 'R', 
                                           'G_L': 60, 'PA_L': 220, 'AB_L': 200, 'H_L': 50, '1B_L': 34, '2B_L': 10, 
                                           '3B_L': 1, 'HR_L': 5, 'R_L': 25, 'RBI_L': 25, 'BB_L': 20, 'IBB_L': 0, 
                                           'SO_L': 50, 'HBP_L': 3, 'SF_L': 2, 'SH_L': 0, 'GDP_L': 3, 'SB_L': 5, 
                                           'CS_L': 1, 'AVG_L': .250, 'G_L': 60, 'PA_R': 220, 'AB_R': 200, 'H_R': 50, 
                                           '1B_R': 34, '2B_R': 10, '3B_R': 1,'HR_R': 5, 'R_R': 25, 'RBI_R': 25, 
                                           'BB_R': 20, 'IBB_R': 0, 'SO_R': 50, 'HBP_R': 3,'SF_R': 2, 'SH_R': 0, 
                                           'GDP_R': 3, 'SB_R': 5, 'CS_R': 1, 'AVG_R': .250})
                        
                        col_headers = stats.index

                    if(batter_df.empty):
                        batter_df = pd.DataFrame(columns=col_headers)

                    batter_df = batter_df.append(stats, ignore_index=True)
         
        pitcher_df = pitcher_df.append(pitcher_stats.loc[pitcher_stats['Name']=='Avg_Totals'])
        batter_df = batter_df.append(batter_stats.loc[batter_stats['Name']=='Avg_Totals'])
        
        team_dict.update({city: Team(city, league[city], batter_df, pitcher_df, starters[city], relievers[city])})
        
    return team_dict

In [182]:
relievers_dict = projected_relievers()

In [183]:
batter_stats_df = reading_data('Splits_Leaderboard_Data_Hitters.csv')
pitcher_stats_df = reading_data('Splits_Leaderboard_Data_Pitchers.csv')
batter_stats_df, pitcher_stats_df = column_type(batter_stats_df, pitcher_stats_df)
starters_dict = projected_starters()
schedule = reading_data('2020_mlb_schedule.csv')
team_dict_df = create_teams(batter_stats_df, pitcher_stats_df, starters_dict, relievers_dict)

TypeError: __init__() missing 2 required positional arguments: 'sb_rate' and 'sb_success'

In [224]:
team_dict_df = create_teams(batter_stats_df, pitcher_stats_df, starters_dict, relievers_dict)

In [155]:
s = Season(team_dict_df['Detroit'], team_dict_df['Washington'])
s.reset_season([team_dict_df['Detroit'], team_dict_df['Washington']])
s.simulate_season(162)
s.print_record()

72-90
90-72


In [225]:
s = FullSeason(schedule)
s.reset_season()
s.simulate_season()
s.print_record()

Atlanta: 69-93
Washington: 82-80
New York Mets: 92-70
Philadelphia: 82-80
Miami: 74-88
St. Louis: 77-85
Milwaukee: 81-81
Chicago Cubs: 87-75
Cincinatti: 89-73
Pittsburg: 64-98
Los Angeles Dodgers: 95-67
Arizona: 97-65
San Francisco: 55-107
Colorado: 68-94
San Diego: 92-70
New York Yankees: 90-72
Tampa Bay: 93-69
Boston: 83-79
Toronto: 76-86
Baltimore: 49-113
Minnesota: 104-58
Cleveland: 78-84
Chicago White Sox: 92-70
Kansas City: 64-98
Detroit: 58-104
Houston: 126-36
Oakland: 93-69
Texas: 77-85
Los Angeles Angels: 89-73
Seattle: 54-108


In [116]:
for acronym, city in acronym_to_city_dict.items():
    away_team = team_dict_df[acronym_to_city_dict[acronym]]
    home_team_acronym = schedule.loc[schedule['date']=='3/28/2020'][acronym].values[0]

    if(not pd.isna(home_team_acronym)):
        home_team = team_dict_df[acronym_to_city_dict[home_team_acronym]]
        g = Game(away_team, home_team)
        g.play() 
        g.print_result()

final score: New York Mets 2 Washington 4
final score: Miami 3 Philadelphia 1
final score: Milwaukee 6 Chicago Cubs 3
final score: Cincinatti 0 St. Louis 5
final score: Los Angeles Dodgers 5 San Francisco 4
final score: Arizona 1 Atlanta 3
final score: San Diego 4 Colorado 2
final score: Tampa Bay 0 Pittsburg 4
final score: Toronto 3 Boston 6
final score: Baltimore 2 New York Yankees 1
final score: Cleveland 1 Detroit 3
final score: Chicago White Sox 10 Kansas City 1
final score: Houston 5 Los Angeles Angels 3
final score: Oakland 4 Minnesota 2
final score: Seattle 3 Texas 2


In [57]:
avg_stats = batter_stats_df.loc[batter_stats_df['Name']=='Avg_Totals']
avg_stats

,Season,Name,playerId,Team,Bats,G_L,PA_L,AB_L,H_L,1B_L,...,BB_R,IBB_R,SO_R,HBP_R,SF_R,SH_R,GDP_R,SB_R,CS_R,AVG_R
529,2019,Avg_Totals,0,NONE,S,47.506616,94.31758,84.213611,22.022684,13.695652,...,20.996219,0.996219,54.482042,2.695652,1.542533,0.582231,4.413989,3.168242,1.094518,0.23899
